In [4]:
%pip install requests beautifulsoup4 langchain sentence-transformers chromadb streamlit


  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached chromadb-1.0.15-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)
  Using cached streamlit-1.46.1-py3-none-any.whl.metadata (9.0 kB)
  Using cached langchain_core-0.3.68-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.4.4-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached sqlalchemy-2.0.41-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached transformers-4.53.1-py3-none-any.whl.metadata (40 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.7.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached scikit_learn-1.7.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2

In [6]:
%pip install langchain_community
# Core
import os
import requests
from bs4 import BeautifulSoup

# LangChain & Embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# For local saving
import pickle


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.8/241.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.1/256.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.4/224.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.6/355.6 kB 7.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
# Set up directories
os.makedirs("data", exist_ok=True)
os.makedirs("embeddings", exist_ok=True)


In [7]:
%pip install lxml
%pip install openai


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 7.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.0/352.0 kB 10.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:


import requests
from bs4 import BeautifulSoup

# List of pages to scrape
urls = {
    "data": "https://mosdac.gov.in/data",
    "satellites": "https://mosdac.gov.in/satellites",
    "live": "https://mosdac.gov.in/live",
    "faq": "https://mosdac.gov.in/faq",
    "about": "https://mosdac.gov.in/about"
}

# Function to extract visible text from a page
def extract_text_from_url(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "lxml")
        # Extract from paragraphs, list items, and table cells
        texts = []
        for tag in ["p", "li", "td"]:
            elements = soup.find_all(tag)
            texts.extend([el.get_text(strip=True) for el in elements if el.get_text(strip=True)])
        return "\n".join(texts)
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return ""

# Scrape all pages
combined_data = ""
for name, url in urls.items():
    print(f"Scraping: {name} ({url})")
    page_text = extract_text_from_url(url)
    combined_data += f"\n\n====== {name.upper()} PAGE ======\n\n" + page_text

# Save to file
with open("mosdac_all_pages.txt", "w", encoding="utf-8") as f:
    f.write(combined_data)

print("\n✅ Done! Scraped content saved to 'mosdac_all_pages.txt'")


Scraping: data (https://mosdac.gov.in/data)
Scraping: satellites (https://mosdac.gov.in/satellites)
Scraping: live (https://mosdac.gov.in/live)
Scraping: faq (https://mosdac.gov.in/faq)
Scraping: about (https://mosdac.gov.in/about)

✅ Done! Scraped content saved to 'mosdac_all_pages.txt'


In [ ]:
import re

# Load scraped file
with open("mosdac_all_pages.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# Define garbage phrases to remove
garbage_patterns = [
    r"(?i)skip to main content",
    r"(?i)login", r"(?i)password", r"(?i)user id",
    r"(?i)sitemap", r"(?i)privacy policy", r"(?i)terms of use",
    r"(?i)home\s*\|.*",  # Navigation bars
    r"\bOK\b", r"\bYes\b", r"\bNo\b"
]

# Remove garbage text
for pattern in garbage_patterns:
    raw_text = re.sub(pattern, '', raw_text)

with open("mosdac_cleaned.txt", "w", encoding="utf-8") as f:
    f.write(raw_text)

print("✅ Cleaned data saved to 'mosdac_cleaned.txt'")


✅ Cleaned data saved to 'mosdac_cleaned.txt'


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# 1. Load Cleaned Text
with open("mosdac_cleaned.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# 2. Split into Chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
chunks = text_splitter.split_text(raw_text)
print(f"✅ Split into {len(chunks)} chunks")

# 3. Initialize Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 4. Store in Chroma
vectorstore = Chroma.from_texts(chunks, embedding=embedding_model, persist_directory="mosdac_chroma")
vectorstore.persist()
print("✅ Embeddings stored in 'mosdac_chroma'")


✅ Split into 29 chunks


/tmp/ipykernel_22615/3178798771.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/home/harji22/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Embeddings stored in 'mosdac_chroma'


/tmp/ipykernel_22615/3178798771.py:22: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub  # or use dummy if no LLM
from langchain.document_loaders import TextLoader
from langchain.chains import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()


# 1. Load Chroma VectorStore
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory="mosdac_chroma", embedding_function=embedding_model)

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a helpful and precise assistant answering questions based only on the provided context.
Be concise and clear in your response. If the answer is not available in the context, reply with:
"I could not find that information in the current data."

Context:
{context}

Question:
{question}

Answer:
"""
)

# 3. Load OpenRouter-compatible LLM (DeepSeek)
llm = ChatOpenAI(
    model="deepseek/deepseek-chat:free",  # Corrected model name for OpenRouter
    temperature=0,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.getenv("OPENROUTER_API_KEY")  # Make sure your key is exported correctly
)

# 4. Create the document chain (StuffDocumentsChain expects LLMChain)
llm_chain = LLMChain(llm=llm, prompt=prompt)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="context")

# 5. Combine with RetrievalQA
qa_chain = RetrievalQA(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    combine_documents_chain=stuff_chain,
    return_source_documents=False
)


# 6. Ask a question
query = "What is MOSDAC and what kind of data does it provide?"
response = qa_chain.run(query)

print("Q:", query)
print("A:", response)


Q: What is MOSDAC and what kind of data does it provide?
A: MOSDAC (Meteorological and Oceanographic Satellite Data Archival Centre) is a website owned and maintained by the Space Applications Centre, Indian Space Research Organisation (ISRO), Government of India. It provides various types of data, including:

- **Atmosphere**: Bayesian-based MT-SAPHIR rainfall, GPS-derived integrated water vapour, GSMap ISRO Rain, METEOSAT8 Cloud Properties.
- **Land**: 3D Volumetric TERLS DWR product, Inland Water Height, River Discharge, Soil Moisture.
- **Ocean**: Global Ocean Surface Current, High-Resolution Sea Surface Salinity, Indian Mainland Coastal Product, Ocean Subsurface, Oceanic Eddies Detection, Sea Ice Occurrence Probability, Wave-based Renewable Energy.
